# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

I = cv2.imread('dom.png') 
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

In [ ]:
def make_frame(I):
    new_I = np.zeros((I.shape[0]+2, I.shape[1]+2))
    new_I[1:-1, 1:-1] = I
    
    new_I[0, 1:-1] = I[-1, :]
    new_I[-1, 1:-1] = I[0, :]
    new_I[1:-1, 0] = I[:, -1]
    new_I[1:-1, -1] = I[:, 0]
    
    new_I[0,0] = I[-1,-1]
    new_I[-1,0] = I[0,-1]
    new_I[0,-1] = I[-1,0]
    new_I[-1,-1] = I[0,0]
    
    return new_I
    

def remove_frame(I):
    return I[1:-1, 1:-1]

def crossing(I, thr):
    I = make_frame(I)
    res = np.zeros_like(I)
    for i in range(1, I.shape[0]-1):
        for j in range(1, I.shape[1]-1):
            surr = I[i-1:i+2, j-1:j+2]
            min_value = np.amin(surr)
            max_value = np.amax(surr)
            if min_value < 0 and max_value > 0:
                if I[i, j] < 0:
                    res[i, j] = np.abs(I[i, j]) + max_value
                elif I[i, j] > 0:
                    res[i, j] = I[i, j] + np.abs(max_value)
    res = remove_frame(res)
    res += np.amin(res) * (-1)
    max_value_res = np.amax(res)
    res *= (255/max_value_res)
    res = (res > thr) * 255
    res = res.astype(np.uint8)
    res = cv2.medianBlur(res, 3)
    return res

In [ ]:
thr = 80
sigma = 5
size = (3, 3)
I_gaussian = cv2.GaussianBlur(I, size, sigma)
I_lapl = cv2.Laplacian(I_gaussian, cv2.CV_32F)
I_res = crossing(I_lapl, thr)

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(I, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()
plt.figure(figsize=(7,7))
plt.imshow(I_res, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w 1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj algorytm detekcji krawędziCanny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
    - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
        Można to wykoanać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować wbardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj algorytm Cannego.

In [ ]:
def nonmax(directions, amplituda):
    res = np.zeros_like(amplituda)
    for i in range(1, res.shape[0]-1):
        for j in range(1, res.shape[1]-1):
            surr = amplituda[i-1:i+2, j-1:j+2]
            direct = directions[i, j]
            if direct == 1:
                values = np.array([surr[1, 0], surr[1, 1], surr[1, 2]])
                max_v = np.amax(values)
                if surr[1, 1] == max_v:
                    res[i, j] = max_v
            elif direct == 2:
                values = np.array([surr[0, 1], surr[1, 1], surr[2, 1]])
                max_v = np.amax(values)
                if surr[1, 1] == max_v:
                    res[i, j] = max_v
            elif direct == 3:
                values = np.array([surr[2, 0], surr[1, 1], surr[0, 2]])
                max_v = np.amax(values)
                if surr[1, 1] == max_v:
                    res[i, j] = max_v
            elif direct == 4:
                values = np.array([surr[0, 0], surr[1, 1], surr[2, 2]])
                max_v = np.amax(values)
                if surr[1, 1] == max_v:
                    res[i, j] = max_v
    return res

def filter_mask (I, mask):
    res = np.zeros_like(I)
    for i in range(1, res.shape[0]-1):
        for j in range(1, res.shape[1]-1):
            surr = I[i-1:i+2, j-1:j+2]
            res[i, j] = np.sum(surr * mask)
    return res

sigma = 5
size = (3, 3)
thr1 = 75
thr2 = 150
I_gaussian = cv2.GaussianBlur(I, size, sigma).astype(int)

S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
S2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

gradient_x = filter_mask(I_gaussian, S1)
gradient_y = filter_mask(I_gaussian, S2)
amplituda = np.sqrt(gradient_x ** 2 + gradient_y ** 2)
angle = np.arctan2(gradient_y, gradient_x)
angle[angle > np.pi/2] -= np.pi
angle[angle < (-1)*np.pi/2] += np.pi

grad_x = gradient_x.copy()
grad_x += ((grad_x == 0) * 1).astype(np.uint8)

step_angle = (22.5/180) * np.pi

mask_1 = ((angle <= step_angle) & (angle >= -1*step_angle)) * 1
mask_2 = ((angle >= (np.pi/2) - step_angle) | (angle <= (-np.pi/2) + step_angle)) * 2
mask_3 = ((angle < 3*step_angle) & (angle > step_angle)) * 3
mask_4 = ((angle < -1*step_angle) & (angle > -3*step_angle)) * 4
classfication_angle = mask_1 + mask_2 + mask_3 + mask_4
nonmax_values = nonmax(classfication_angle, amplituda)

gnh = nonmax_values >= thr2
gnl = (nonmax_values >= thr1) & (nonmax_values < thr2)
stack = list(np.argwhere(gnh))
visited = np.zeros_like(gnh)
result = np.zeros_like(gnh)
while stack:
    x, y = stack.pop()
    if visited[x, y]:
        continue
    visited[x, y] = 1
    result[x, y] = 1
    for i in range(max(x-1, 0), min(x+2, result.shape[0])):
        for j in range(max(y-1, 0), min(y+2, result.shape[1])):
            if gnl[i, j]:
                stack.append([i, j])

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(I, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()
plt.figure(figsize=(7,7))
plt.imshow(gnh, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()
plt.figure(figsize=(7,7))
plt.imshow(result, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z własną implementacją.

In [ ]:
I_res = cv2.Canny(I, 200, 350, None, 3, 1)

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(I_res, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()